# פרק 10: אינטראופרביליות עם ספריות Python אחרות

## מבוא

אחד היתרונות הגדולים של Polars הוא היכולת שלו לעבוד בצורה חלקה עם ספריות Python פופולריות אחרות. במדריך זה נלמד כיצד:

- ✅ להמיר בין Polars ל-pandas ובחזרה
- ✅ לעבוד עם NumPy arrays
- ✅ להשתמש ב-PyArrow לעיבוד מהיר
- ✅ לשלב עם DuckDB לשאילתות SQL חזקות

---

## 📦 התקנת הספריות הנדרשות

הריצו את התא הבא כדי לוודא שכל הספריות מותקנות:

In [1]:
# התקנת כל הספריות הנדרשות
# !pip install polars pandas numpy pyarrow duckdb

In [3]:
# ייבוא הספריות
import polars as pl
import pandas as pd
import numpy as np
import pyarrow as pa
import duckdb

print(f"Polars version: {pl.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Polars version: 1.34.0
Pandas version: 2.3.0
NumPy version: 2.2.0


---

## 🐼 חלק 1: המרה בין Polars ל-pandas

### מדוע זה חשוב?

pandas היא הספרייה הפופולרית ביותר לעיבוד נתונים ב-Python. למרות ש-Polars מהיר יותר, לפעמים נצטרך:
- לעבוד עם קוד קיים שכתוב ב-pandas
- להשתמש בספריות שתומכות רק ב-pandas
- לשתף נתונים עם עמיתים שעובדים עם pandas

### 1.1 יצירת DataFrame ב-Polars

In [4]:
# יצירת DataFrame פשוט
df_polars = pl.DataFrame({
    'שם': ['דני', 'שרה', 'יוסי', 'מיכל'],
    'גיל': [25, 30, 35, 28],
    'עיר': ['תל אביב', 'חיפה', 'ירושלים', 'באר שבע'],
    'שכר': [10000, 15000, 12000, 11000]
})

print("🔵 Polars DataFrame:")
print(df_polars)
print(f"\nסוג האובייקט: {type(df_polars)}")

🔵 Polars DataFrame:
shape: (4, 4)
┌──────┬─────┬─────────┬───────┐
│ שם   ┆ גיל ┆ עיר     ┆ שכר   │
│ ---  ┆ --- ┆ ---     ┆ ---   │
│ str  ┆ i64 ┆ str     ┆ i64   │
╞══════╪═════╪═════════╪═══════╡
│ דני  ┆ 25  ┆ תל אביב ┆ 10000 │
│ שרה  ┆ 30  ┆ חיפה    ┆ 15000 │
│ יוסי ┆ 35  ┆ ירושלים ┆ 12000 │
│ מיכל ┆ 28  ┆ באר שבע ┆ 11000 │
└──────┴─────┴─────────┴───────┘

סוג האובייקט: <class 'polars.dataframe.frame.DataFrame'>


### 1.2 המרה מ-Polars ל-pandas

In [5]:
# המרה ל-pandas DataFrame
df_pandas = df_polars.to_pandas()

print("🟠 pandas DataFrame:")
print(df_pandas)
print(f"\nסוג האובייקט: {type(df_pandas)}")
print(f"\nטיפוסי הנתונים:\n{df_pandas.dtypes}")

🟠 pandas DataFrame:
     שם  גיל      עיר    שכר
0   דני   25  תל אביב  10000
1   שרה   30     חיפה  15000
2  יוסי   35  ירושלים  12000
3  מיכל   28  באר שבע  11000

סוג האובייקט: <class 'pandas.core.frame.DataFrame'>

טיפוסי הנתונים:
שם     object
גיל     int64
עיר    object
שכר     int64
dtype: object


### 💡 טיפ חשוב: שימוש ב-PyArrow Extension Arrays

כאשר ממירים ל-pandas, אפשר להשתמש ב-PyArrow extension arrays שמשפרים את הביצועים ומשמרים את טיפוסי הנתונים של Polars טוב יותר.

In [6]:
# המרה עם PyArrow extension arrays
df_pandas_arrow = df_polars.to_pandas(use_pyarrow_extension_array=True)

print("טיפוסי נתונים רגילים ב-pandas:")
print(df_pandas.dtypes)

print("\nטיפוסי נתונים עם PyArrow:")
print(df_pandas_arrow.dtypes)

טיפוסי נתונים רגילים ב-pandas:
שם     object
גיל     int64
עיר    object
שכר     int64
dtype: object

טיפוסי נתונים עם PyArrow:
שם     large_string[pyarrow]
גיל           int64[pyarrow]
עיר    large_string[pyarrow]
שכר           int64[pyarrow]
dtype: object


### 1.3 המרה מ-pandas ל-Polars

In [7]:
# שתי דרכים להמיר pandas ל-Polars:

# דרך 1: שימוש ב-pl.from_pandas()
df_back_1 = pl.from_pandas(df_pandas)

# דרך 2: שימוש ב-pl.DataFrame()
df_back_2 = pl.DataFrame(df_pandas)

print("✅ חזרנו ל-Polars!")
print(df_back_1)
print(f"\nסוג האובייקט: {type(df_back_1)}")

✅ חזרנו ל-Polars!
shape: (4, 4)
┌──────┬─────┬─────────┬───────┐
│ שם   ┆ גיל ┆ עיר     ┆ שכר   │
│ ---  ┆ --- ┆ ---     ┆ ---   │
│ str  ┆ i64 ┆ str     ┆ i64   │
╞══════╪═════╪═════════╪═══════╡
│ דני  ┆ 25  ┆ תל אביב ┆ 10000 │
│ שרה  ┆ 30  ┆ חיפה    ┆ 15000 │
│ יוסי ┆ 35  ┆ ירושלים ┆ 12000 │
│ מיכל ┆ 28  ┆ באר שבע ┆ 11000 │
└──────┴─────┴─────────┴───────┘

סוג האובייקט: <class 'polars.dataframe.frame.DataFrame'>


### 1.4 עבודה עם Series

גם Series (עמודה בודדת) ניתנים להמרה בקלות:

In [8]:
# יצירת Polars Series
s_polars = pl.Series('מספרים', [10, 20, 30, 40, 50])

# המרה ל-pandas Series
s_pandas = s_polars.to_pandas()

# המרה חזרה ל-Polars
s_back = pl.from_pandas(s_pandas)

print("Polars Series:")
print(s_polars)
print(f"\nסוג: {type(s_polars)}")

print("\npandas Series:")
print(s_pandas)
print(f"\nסוג: {type(s_pandas)}")

Polars Series:
shape: (5,)
Series: 'מספרים' [i64]
[
	10
	20
	30
	40
	50
]

סוג: <class 'polars.series.series.Series'>

pandas Series:
0    10
1    20
2    30
3    40
4    50
Name: מספרים, dtype: int64

סוג: <class 'pandas.core.series.Series'>


### 🎯 תרגיל 1: המרות בסיסיות

1. צרו Polars DataFrame עם מידע על 3 ספרים (שם, מחבר, מחיר, שנה)
2. המירו אותו ל-pandas
3. הוסיפו עמודה חדשה ב-pandas (למשל: 'קראתי')
4. המירו בחזרה ל-Polars

In [ ]:
# 💻 כתבו את הקוד שלכם כאן:

# 1. צרו DataFrame
books_polars = pl.DataFrame({
    # הוסיפו את העמודות כאן
})

# 2. המירו ל-pandas


# 3. הוסיפו עמודה


# 4. המירו בחזרה


---

## 🔢 חלק 2: עבודה עם NumPy Arrays

### מדוע NumPy?

NumPy היא הספרייה הבסיסית לחישובים מספריים ב-Python. שילוב בין Polars ל-NumPy מאפשר:
- שימוש בפונקציות מתמטיות מתקדמות
- עבודה עם מערכים רב-ממדיים
- ביצועים מעולים בפעולות וקטוריות

### 2.1 המרה מ-NumPy Array ל-Polars DataFrame

In [9]:
# יצירת NumPy array דו-ממדי
arr = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
    [10, 11, 12]
])

print("NumPy Array:")
print(arr)
print(f"Shape: {arr.shape}")

NumPy Array:
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
Shape: (4, 3)


In [11]:
# המרה ל-Polars - כל עמודה ב-array הופכת לעמודה ב-DataFrame
# orient='col' אומר שכל עמודה ב-array היא עמודה ב-DataFrame
df_from_numpy = pl.from_numpy(arr, schema=['א', 'ב', 'ג','ד'], orient='col')

print("\nPolars DataFrame (orient='col'):")
print(df_from_numpy)


Polars DataFrame (orient='col'):
shape: (3, 4)
┌─────┬─────┬─────┬─────┐
│ א   ┆ ב   ┆ ג   ┆ ד   │
│ --- ┆ --- ┆ --- ┆ --- │
│ i64 ┆ i64 ┆ i64 ┆ i64 │
╞═════╪═════╪═════╪═════╡
│ 1   ┆ 4   ┆ 7   ┆ 10  │
│ 2   ┆ 5   ┆ 8   ┆ 11  │
│ 3   ┆ 6   ┆ 9   ┆ 12  │
└─────┴─────┴─────┴─────┘


In [12]:
# orient='row' - כל שורה ב-array היא שורה ב-DataFrame
df_from_numpy_row = pl.from_numpy(arr, schema=['עמודה_1', 'עמודה_2', 'עמודה_3'], orient='row')

print("Polars DataFrame (orient='row'):")
print(df_from_numpy_row)

Polars DataFrame (orient='row'):
shape: (4, 3)
┌─────────┬─────────┬─────────┐
│ עמודה_1 ┆ עמודה_2 ┆ עמודה_3 │
│ ---     ┆ ---     ┆ ---     │
│ i64     ┆ i64     ┆ i64     │
╞═════════╪═════════╪═════════╡
│ 1       ┆ 2       ┆ 3       │
│ 4       ┆ 5       ┆ 6       │
│ 7       ┆ 8       ┆ 9       │
│ 10      ┆ 11      ┆ 12      │
└─────────┴─────────┴─────────┘


### 💡 הבדל בין orient='col' ל-orient='row'

- **orient='col'**: כל עמודה ב-NumPy array → עמודה ב-DataFrame
- **orient='row'**: כל שורה ב-NumPy array → שורה ב-DataFrame

### 2.2 המרה מ-Polars ל-NumPy

In [13]:
# יצירת DataFrame
df = pl.DataFrame({
    'x': [1, 2, 3, 4],
    'y': [10, 20, 30, 40],
    'z': [100, 200, 300, 400]
})

print("Polars DataFrame:")
print(df)

Polars DataFrame:
shape: (4, 3)
┌─────┬─────┬─────┐
│ x   ┆ y   ┆ z   │
│ --- ┆ --- ┆ --- │
│ i64 ┆ i64 ┆ i64 │
╞═════╪═════╪═════╡
│ 1   ┆ 10  ┆ 100 │
│ 2   ┆ 20  ┆ 200 │
│ 3   ┆ 30  ┆ 300 │
│ 4   ┆ 40  ┆ 400 │
└─────┴─────┴─────┘


In [14]:
# המרה רגילה ל-NumPy array
arr_standard = df.to_numpy()

print("\nNumPy Array (רגיל):")
print(arr_standard)
print(f"Shape: {arr_standard.shape}")
print(f"dtype: {arr_standard.dtype}")


NumPy Array (רגיל):
[[  1  10 100]
 [  2  20 200]
 [  3  30 300]
 [  4  40 400]]
Shape: (4, 3)
dtype: int64


In [15]:
# המרה ל-structured array (שומר שמות עמודות)
arr_structured = df.to_numpy(structured=True)

print("\nNumPy Structured Array:")
print(arr_structured)
print(f"\ndtype: {arr_structured.dtype}")

# ניתן לגשת לעמודות לפי שם:
print(f"\nעמודה 'x': {arr_structured['x']}")


NumPy Structured Array:
[(1, 10, 100) (2, 20, 200) (3, 30, 300) (4, 40, 400)]

dtype: [('x', '<i8'), ('y', '<i8'), ('z', '<i8')]

עמודה 'x': [1 2 3 4]


### 2.3 שימוש בפונקציות NumPy על Polars

In [16]:
# יצירת DataFrame עם מספרים
df_math = pl.DataFrame({
    'a': [12, 18, 24, 36, 48],
    'b': [8, 12, 16, 24, 32]
})

print("DataFrame המקורי:")
print(df_math)

DataFrame המקורי:
shape: (5, 2)
┌─────┬─────┐
│ a   ┆ b   │
│ --- ┆ --- │
│ i64 ┆ i64 │
╞═════╪═════╡
│ 12  ┆ 8   │
│ 18  ┆ 12  │
│ 24  ┆ 16  │
│ 36  ┆ 24  │
│ 48  ┆ 32  │
└─────┴─────┘


In [17]:
# שימוש ב-NumPy gcd (Greatest Common Divisor - מחלק משותף מקסימלי)
df_with_gcd = df_math.with_columns(
    np.gcd(pl.col('a'), pl.col('b')).alias('מחלק_משותף')
)

print("\nלאחר הוספת המחלק המשותף:")
print(df_with_gcd)


לאחר הוספת המחלק המשותף:
shape: (5, 3)
┌─────┬─────┬────────────┐
│ a   ┆ b   ┆ מחלק_משותף │
│ --- ┆ --- ┆ ---        │
│ i64 ┆ i64 ┆ i64        │
╞═════╪═════╪════════════╡
│ 12  ┆ 8   ┆ 4          │
│ 18  ┆ 12  ┆ 6          │
│ 24  ┆ 16  ┆ 8          │
│ 36  ┆ 24  ┆ 12         │
│ 48  ┆ 32  ┆ 16         │
└─────┴─────┴────────────┘


In [18]:
# דוגמאות נוספות לפונקציות NumPy
df_advanced = df_math.with_columns([
    np.sqrt(pl.col('a')).alias('שורש_a'),
    np.power(pl.col('b'), 2).alias('b_בריבוע'),
    np.log(pl.col('a')).alias('לוג_a')
])

print("\nעם פונקציות מתמטיות נוספות:")
print(df_advanced)


עם פונקציות מתמטיות נוספות:
shape: (5, 5)
┌─────┬─────┬──────────┬──────────┬──────────┐
│ a   ┆ b   ┆ שורש_a   ┆ b_בריבוע ┆ לוג_a    │
│ --- ┆ --- ┆ ---      ┆ ---      ┆ ---      │
│ i64 ┆ i64 ┆ f64      ┆ i64      ┆ f64      │
╞═════╪═════╪══════════╪══════════╪══════════╡
│ 12  ┆ 8   ┆ 3.464102 ┆ 64       ┆ 2.484907 │
│ 18  ┆ 12  ┆ 4.242641 ┆ 144      ┆ 2.890372 │
│ 24  ┆ 16  ┆ 4.898979 ┆ 256      ┆ 3.178054 │
│ 36  ┆ 24  ┆ 6.0      ┆ 576      ┆ 3.583519 │
│ 48  ┆ 32  ┆ 6.928203 ┆ 1024     ┆ 3.871201 │
└─────┴─────┴──────────┴──────────┴──────────┘


### 2.4 המרת Series ל-NumPy

In [19]:
# יצירת Series
s = pl.Series('טמפרטורות', [20, 25, 30, 22, 28, 35])

print("Polars Series:")
print(s)

# המרה ל-NumPy array
arr_from_series = s.to_numpy()

print("\nNumPy Array:")
print(arr_from_series)

# שימוש בפונקציות NumPy
print(f"\nממוצע: {np.mean(arr_from_series):.2f}")
print(f"סטיית תקן: {np.std(arr_from_series):.2f}")
print(f"מינימום: {np.min(arr_from_series)}")
print(f"מקסימום: {np.max(arr_from_series)}")

Polars Series:
shape: (6,)
Series: 'טמפרטורות' [i64]
[
	20
	25
	30
	22
	28
	35
]

NumPy Array:
[20 25 30 22 28 35]

ממוצע: 26.67
סטיית תקן: 5.02
מינימום: 20
מקסימום: 35


### 🎯 תרגיל 2: חישובים עם NumPy

1. צרו DataFrame עם שתי עמודות של מספרים (5 שורות)
2. המירו ל-NumPy array
3. חשבו את הממוצע של כל עמודה
4. הוסיפו עמודה חדשה שמכילה את הסכום של שתי העמודות המקוריות
5. הוסיפו עמודה נוספת עם המכפלה של העמודות

In [ ]:
# 💻 כתבו את הקוד שלכם כאן:

# 1. צרו DataFrame


# 2. המירו ל-NumPy


# 3. חשבו ממוצעים


# 4-5. הוסיפו עמודות חדשות


---

## 🏹 חלק 3: אינטגרציה עם PyArrow

### מה זה PyArrow?

PyArrow הוא ממשק Python ל-Apache Arrow - פורמט זיכרון עמודתי שמאפשר:
- ✅ קריאה וכתיבה מהירה מאוד של קבצים
- ✅ עבודה יעילה עם נתונים גדולים
- ✅ שיתוף זיכרון בין תהליכים
- ✅ תמיכה בפורמטים כמו Parquet

Polars בנוי על PyArrow, אז האינטגרציה ביניהם מושלמת!

### 3.1 המרה בין Polars ל-PyArrow Table

In [20]:
# יצירת Polars DataFrame
df_polars = pl.DataFrame({
    'מוצר': ['מחשב נייד', 'עכבר', 'מקלדת', 'מסך'],
    'מחיר': [3500, 120, 250, 1500],
    'מלאי': [15, 45, 30, 20]
})

print("Polars DataFrame:")
print(df_polars)

Polars DataFrame:
shape: (4, 3)
┌───────────┬──────┬──────┐
│ מוצר      ┆ מחיר ┆ מלאי │
│ ---       ┆ ---  ┆ ---  │
│ str       ┆ i64  ┆ i64  │
╞═══════════╪══════╪══════╡
│ מחשב נייד ┆ 3500 ┆ 15   │
│ עכבר      ┆ 120  ┆ 45   │
│ מקלדת     ┆ 250  ┆ 30   │
│ מסך       ┆ 1500 ┆ 20   │
└───────────┴──────┴──────┘


In [21]:
# המרה ל-PyArrow Table
arrow_table = df_polars.to_arrow()

print("\nPyArrow Table:")
print(arrow_table)
print(f"\nסוג: {type(arrow_table)}")


PyArrow Table:
pyarrow.Table
מוצר: large_string
מחיר: int64
מלאי: int64
----
מוצר: [["מחשב נייד","עכבר","מקלדת","מסך"]]
מחיר: [[3500,120,250,1500]]
מלאי: [[15,45,30,20]]

סוג: <class 'pyarrow.lib.Table'>


In [22]:
# המרה חזרה מ-PyArrow ל-Polars
df_back = pl.from_arrow(arrow_table)

print("\nחזרנו ל-Polars:")
print(df_back)


חזרנו ל-Polars:
shape: (4, 3)
┌───────────┬──────┬──────┐
│ מוצר      ┆ מחיר ┆ מלאי │
│ ---       ┆ ---  ┆ ---  │
│ str       ┆ i64  ┆ i64  │
╞═══════════╪══════╪══════╡
│ מחשב נייד ┆ 3500 ┆ 15   │
│ עכבר      ┆ 120  ┆ 45   │
│ מקלדת     ┆ 250  ┆ 30   │
│ מסך       ┆ 1500 ┆ 20   │
└───────────┴──────┴──────┘


### 3.2 עבודה עם PyArrow Datasets (קבצים מחולקים)

In [23]:
# דוגמה ליצירת dataset מחולק (Partitioned)
# זה שימושי כשיש לנו קבצים רבים בתיקיות שונות

import pyarrow.dataset as ds

# יצירת נתוני דוגמה
df_sales = pl.DataFrame({
    'תאריך': ['2024-01-15', '2024-01-20', '2024-02-10', '2024-02-15'],
    'מוצר': ['מחשב', 'טלפון', 'מחשב', 'טאבלט'],
    'סכום': [5000, 3000, 4500, 2000],
    'חודש': [1, 1, 2, 2]
})

print("DataFrame המכירות:")
print(df_sales)

DataFrame המכירות:
shape: (4, 4)
┌────────────┬───────┬──────┬──────┐
│ תאריך      ┆ מוצר  ┆ סכום ┆ חודש │
│ ---        ┆ ---   ┆ ---  ┆ ---  │
│ str        ┆ str   ┆ i64  ┆ i64  │
╞════════════╪═══════╪══════╪══════╡
│ 2024-01-15 ┆ מחשב  ┆ 5000 ┆ 1    │
│ 2024-01-20 ┆ טלפון ┆ 3000 ┆ 1    │
│ 2024-02-10 ┆ מחשב  ┆ 4500 ┆ 2    │
│ 2024-02-15 ┆ טאבלט ┆ 2000 ┆ 2    │
└────────────┴───────┴──────┴──────┘


### 3.3 סריאליזציה מהירה עם PyArrow

In [24]:
# PyArrow מאפשר שמירה וטעינה מהירות
# זה מהיר הרבה יותר מ-CSV!

# שמירה בפורמט Parquet (דרך PyArrow)
df_polars.write_parquet('/tmp/data.parquet')

# קריאה חזרה
df_loaded = pl.read_parquet('/tmp/data.parquet')

print("הנתונים נטענו בהצלחה:")
print(df_loaded)

הנתונים נטענו בהצלחה:
shape: (4, 3)
┌───────────┬──────┬──────┐
│ מוצר      ┆ מחיר ┆ מלאי │
│ ---       ┆ ---  ┆ ---  │
│ str       ┆ i64  ┆ i64  │
╞═══════════╪══════╪══════╡
│ מחשב נייד ┆ 3500 ┆ 15   │
│ עכבר      ┆ 120  ┆ 45   │
│ מקלדת     ┆ 250  ┆ 30   │
│ מסך       ┆ 1500 ┆ 20   │
└───────────┴──────┴──────┘


### 3.4 המרת Series ל-PyArrow Array

In [25]:
# יצירת Series
s = pl.Series('ציונים', [85, 92, 78, 95, 88, 90])

print("Polars Series:")
print(s)

# המרה ל-PyArrow Array
arrow_array = s.to_arrow()

print("\nPyArrow Array:")
print(arrow_array)
print(f"\nType: {arrow_array.type}")

Polars Series:
shape: (6,)
Series: 'ציונים' [i64]
[
	85
	92
	78
	95
	88
	90
]

PyArrow Array:
[
  85,
  92,
  78,
  95,
  88,
  90
]

Type: int64


### 🎯 תרגיל 3: עבודה עם PyArrow

1. צרו DataFrame עם מידע על עובדים (שם, משרה, שכר, מחלקה)
2. המירו ל-PyArrow Table
3. שמרו את הטבלה לקובץ Parquet
4. טענו אותה בחזרה
5. בדקו שהנתונים זהים

In [ ]:
# 💻 כתבו את הקוד שלכם כאן:

# 1. צרו DataFrame


# 2. המירו ל-PyArrow


# 3. שמרו לקובץ


# 4. טענו בחזרה


# 5. בדקו זהות


---

## 🦆 חלק 4: אינטגרציה עם DuckDB

### מה זה DuckDB?

DuckDB הוא מסד נתונים אנליטי מהיר במיוחד שמאפשר:
- ✅ שאילתות SQL על DataFrame-ים
- ✅ ביצועים מעולים על נתונים גדולים
- ✅ שילוב מושלם עם Polars
- ✅ עבודה בזיכרון (in-memory)

### 4.1 שאילתות SQL על Polars DataFrame

In [26]:
# יצירת DataFrame
df_employees = pl.DataFrame({
    'שם': ['דני', 'שרה', 'יוסי', 'מיכל', 'אורי', 'רחל'],
    'מחלקה': ['IT', 'מכירות', 'IT', 'שיווק', 'מכירות', 'שיווק'],
    'שכר': [15000, 12000, 18000, 14000, 13000, 16000],
    'ותק': [3, 5, 7, 2, 4, 6]
})

print("DataFrame העובדים:")
print(df_employees)

DataFrame העובדים:
shape: (6, 4)
┌──────┬────────┬───────┬─────┐
│ שם   ┆ מחלקה  ┆ שכר   ┆ ותק │
│ ---  ┆ ---    ┆ ---   ┆ --- │
│ str  ┆ str    ┆ i64   ┆ i64 │
╞══════╪════════╪═══════╪═════╡
│ דני  ┆ IT     ┆ 15000 ┆ 3   │
│ שרה  ┆ מכירות ┆ 12000 ┆ 5   │
│ יוסי ┆ IT     ┆ 18000 ┆ 7   │
│ מיכל ┆ שיווק  ┆ 14000 ┆ 2   │
│ אורי ┆ מכירות ┆ 13000 ┆ 4   │
│ רחל  ┆ שיווק  ┆ 16000 ┆ 6   │
└──────┴────────┴───────┴─────┘


In [27]:
# שימוש ב-DuckDB לשאילתת SQL
# שימו לב: DuckDB יכול "לראות" את ה-DataFrame ישירות!

result = duckdb.sql("""
    SELECT 
        מחלקה,
        COUNT(*) as מספר_עובדים,
        AVG(שכר) as שכר_ממוצע,
        MAX(שכר) as שכר_מקסימלי
    FROM df_employees
    GROUP BY מחלקה
    ORDER BY שכר_ממוצע DESC
""")

print("\nתוצאות השאילתה (DuckDB):")
result.show()


תוצאות השאילתה (DuckDB):
┌─────────┬─────────────┬───────────┬─────────────┐
│  מחלקה  │ מספר_עובדים │ שכר_ממוצע │ שכר_מקסימלי │
│ varchar │    int64    │  double   │    int64    │
├─────────┼─────────────┼───────────┼─────────────┤
│ IT      │           2 │   16500.0 │       18000 │
│ שיווק   │           2 │   15000.0 │       16000 │
│ מכירות  │           2 │   12500.0 │       13000 │
└─────────┴─────────────┴───────────┴─────────────┘



In [28]:
# המרה של תוצאות DuckDB חזרה ל-Polars
df_result = result.pl()

print("\nתוצאות ב-Polars DataFrame:")
print(df_result)
print(f"\nסוג: {type(df_result)}")


תוצאות ב-Polars DataFrame:
shape: (3, 4)
┌────────┬─────────────┬───────────┬─────────────┐
│ מחלקה  ┆ מספר_עובדים ┆ שכר_ממוצע ┆ שכר_מקסימלי │
│ ---    ┆ ---         ┆ ---       ┆ ---         │
│ str    ┆ i64         ┆ f64       ┆ i64         │
╞════════╪═════════════╪═══════════╪═════════════╡
│ IT     ┆ 2           ┆ 16500.0   ┆ 18000       │
│ שיווק  ┆ 2           ┆ 15000.0   ┆ 16000       │
│ מכירות ┆ 2           ┆ 12500.0   ┆ 13000       │
└────────┴─────────────┴───────────┴─────────────┘

סוג: <class 'polars.dataframe.frame.DataFrame'>


### 4.2 שאילתות מורכבות יותר

In [29]:
# שאילתה עם תנאים מורכבים
high_earners = duckdb.sql("""
    SELECT 
        שם,
        מחלקה,
        שכר,
        CASE 
            WHEN שכר >= 16000 THEN 'גבוה'
            WHEN שכר >= 14000 THEN 'בינוני'
            ELSE 'נמוך'
        END as רמת_שכר
    FROM df_employees
    WHERE ותק >= 3
    ORDER BY שכר DESC
""")

print("עובדים ותיקים לפי רמת שכר:")
high_earners.show()

עובדים ותיקים לפי רמת שכר:
┌─────────┬─────────┬───────┬─────────┐
│   שם    │  מחלקה  │  שכר  │ רמת_שכר │
│ varchar │ varchar │ int64 │ varchar │
├─────────┼─────────┼───────┼─────────┤
│ יוסי    │ IT      │ 18000 │ גבוה    │
│ רחל     │ שיווק   │ 16000 │ גבוה    │
│ דני     │ IT      │ 15000 │ בינוני  │
│ אורי    │ מכירות  │ 13000 │ נמוך    │
│ שרה     │ מכירות  │ 12000 │ נמוך    │
└─────────┴─────────┴───────┴─────────┘



### 4.3 JOIN בין DataFrames עם DuckDB

In [30]:
# יצירת DataFrame נוסף עם פרויקטים
df_projects = pl.DataFrame({
    'עובד': ['דני', 'שרה', 'יוסי', 'דני', 'מיכל'],
    'פרויקט': ['אתר חדש', 'קמפיין', 'מערכת CRM', 'אפליקציה', 'קטלוג'],
    'שעות': [120, 80, 150, 100, 90]
})

print("DataFrame הפרויקטים:")
print(df_projects)

DataFrame הפרויקטים:
shape: (5, 3)
┌──────┬───────────┬──────┐
│ עובד ┆ פרויקט    ┆ שעות │
│ ---  ┆ ---       ┆ ---  │
│ str  ┆ str       ┆ i64  │
╞══════╪═══════════╪══════╡
│ דני  ┆ אתר חדש   ┆ 120  │
│ שרה  ┆ קמפיין    ┆ 80   │
│ יוסי ┆ מערכת CRM ┆ 150  │
│ דני  ┆ אפליקציה  ┆ 100  │
│ מיכל ┆ קטלוג     ┆ 90   │
└──────┴───────────┴──────┘


In [31]:
# JOIN בין שני ה-DataFrames
joined = duckdb.sql("""
    SELECT 
        e.שם,
        e.מחלקה,
        e.שכר,
        p.פרויקט,
        p.שעות
    FROM df_employees e
    INNER JOIN df_projects p ON e.שם = p.עובד
    ORDER BY e.שם, p.שעות DESC
""")

print("\nעובדים והפרויקטים שלהם:")
joined.show()


עובדים והפרויקטים שלהם:
┌─────────┬─────────┬───────┬───────────┬───────┐
│   שם    │  מחלקה  │  שכר  │  פרויקט   │ שעות  │
│ varchar │ varchar │ int64 │  varchar  │ int64 │
├─────────┼─────────┼───────┼───────────┼───────┤
│ דני     │ IT      │ 15000 │ אתר חדש   │   120 │
│ דני     │ IT      │ 15000 │ אפליקציה  │   100 │
│ יוסי    │ IT      │ 18000 │ מערכת CRM │   150 │
│ מיכל    │ שיווק   │ 14000 │ קטלוג     │    90 │
│ שרה     │ מכירות  │ 12000 │ קמפיין    │    80 │
└─────────┴─────────┴───────┴───────────┴───────┘



### 4.4 חישובים סטטיסטיים עם DuckDB

In [ ]:
# חישובים סטטיסטיים מתקדמים
stats = duckdb.sql("""
    SELECT 
        COUNT(*) as סה_כ_עובדים,
        AVG(שכר) as שכר_ממוצע,
        MEDIAN(שכר) as חציון_שכר,
        STDDEV(שכר) as סטיית_תקן,
        MIN(שכר) as שכר_מינימום,
        MAX(שכר) as שכר_מקסימום,
        SUM(שכר) as סכום_משכורות
    FROM df_employees
""")

print("סטטיסטיקות כלליות:")
stats.show()

### 💡 למה להשתמש ב-DuckDB?

**יתרונות:**
- 🚀 מהיר מאוד על נתונים גדולים
- 📝 תחביר SQL מוכר ונוח
- 🔗 אין צורך להמיר נתונים - עובד ישירות על Polars
- 💪 תומך בשאילתות מורכבות (window functions, CTEs, וכו')

**מתי להשתמש:**
- כשאתם נוחים יותר עם SQL מאשר עם Polars expressions
- כשצריך לבצע JOINs מורכבים
- כשיש שאילתות SQL קיימות שרוצים להריץ על DataFrame

### 🎯 תרגיל 4: שאילתות SQL עם DuckDB

צרו שני DataFrames:
1. **לקוחות**: מזהה, שם, עיר, גיל
2. **הזמנות**: מזהה_הזמנה, מזהה_לקוח, סכום, תאריך

בצעו:
1. JOIN בין הטבלאות
2. חשבו את סכום ההזמנות לכל לקוח
3. מצאו את הלקוחות עם ההזמנות הגבוהות ביותר
4. חשבו סטטיסטיקות לפי עיר

In [ ]:
# 💻 כתבו את הקוד שלכם כאן:

# 1. צרו DataFrame של לקוחות
df_customers = pl.DataFrame({
    # הוסיפו את העמודות
})

# 2. צרו DataFrame של הזמנות
df_orders = pl.DataFrame({
    # הוסיפו את העמודות
})

# 3. בצעו JOIN


# 4. חשבו סטטיסטיקות


---

## 📊 סיכום וטבלת השוואה

### מתי להשתמש בכל ספרייה?

| ספרייה | מתי להשתמש | יתרונות עיקריים |
|--------|------------|------------------|
| **pandas** | • עבודה עם קוד קיים<br>• ספריות שתומכות רק ב-pandas<br>• נתונים קטנים-בינוניים | • פופולרית מאוד<br>• תיעוד עשיר<br>• קהילה גדולה |
| **NumPy** | • חישובים מתמטיים מתקדמים<br>• אלגברה לינארית<br>• עבודה עם מערכים רב-ממדיים | • ביצועים מעולים<br>• פונקציות מתמטיות רבות<br>• תקן בתעשייה |
| **PyArrow** | • עבודה עם קבצים גדולים<br>• פורמט Parquet<br>• שיתוף נתונים בין תהליכים | • מהיר ביותר לקריאה/כתיבה<br>• חיסכון במקום<br>• תקן תעשייתי |
| **DuckDB** | • שאילתות SQL מורכבות<br>• JOINs מרובים<br>• אנליטיקה מתקדמת | • תחביר SQL מוכר<br>• ביצועים מעולים<br>• אינטגרציה מושלמת |

### סיכום פקודות המרה

```python
# Polars ↔ pandas
df_pandas = df_polars.to_pandas()
df_polars = pl.from_pandas(df_pandas)

# Polars ↔ NumPy
arr = df.to_numpy()
df = pl.from_numpy(arr, schema=['col1', 'col2'])

# Polars ↔ PyArrow
table = df.to_arrow()
df = pl.from_arrow(table)

# Polars ↔ DuckDB
result = duckdb.sql('SELECT * FROM df')
df_result = result.pl()
```

---

## 🎓 תרגיל מסכם גדול

צרו workflow מלא שמשלב את כל הספריות:

1. צרו DataFrame ב-Polars עם נתוני מכירות
2. המירו ל-pandas והוסיפו עמודה חדשה
3. המירו ל-NumPy וחשבו סטטיסטיקות
4. שמרו לקובץ Parquet דרך PyArrow
5. טענו בחזרה והריצו שאילתת SQL עם DuckDB
6. המירו את התוצאה חזרה ל-Polars

In [ ]:
# 💻 הפתרון המלא שלכם:

# 1. יצירת DataFrame


# 2. עבודה עם pandas


# 3. חישובים עם NumPy


# 4. שמירה עם PyArrow


# 5. שאילתת SQL עם DuckDB


# 6. חזרה ל-Polars


---

## 🎉 סיימתם!

כל הכבוד! עכשיו אתם יודעים:
- ✅ להמיר בין Polars לכל הספריות העיקריות
- ✅ לבחור את הכלי הנכון לכל משימה
- ✅ לשלב ספריות שונות ב-workflow אחד
- ✅ לנצל את היתרונות של כל ספרייה

### 📚 משאבים נוספים:
- [Polars Documentation](https://docs.pola.rs/)
- [pandas Documentation](https://pandas.pydata.org/docs/)
- [NumPy Documentation](https://numpy.org/doc/)
- [PyArrow Documentation](https://arrow.apache.org/docs/python/)
- [DuckDB Documentation](https://duckdb.org/docs/)

### 💡 טיפ אחרון:
תמיד התחילו ב-Polars לעיבוד הנתונים, והמירו לספריות אחרות רק כשבאמת צריך!